In [ ]:
library(plyr)
library(dplyr)
library(stringr)
library(lubridate)
library(ggplot2)

options(repr.matrix.max.cols=30)

Plotting tools in R
------------------

We've spent a lot of time with old-school plotting frameworks in R, in particular plot() and boxplot() and hist(). Today we'll see how to customize these and also provide a few other frameworks that "expose" their functionality a bit more cleanly. As we will see, there are several graphics packages in R. 

Why? Over the years, different approaches for how to structure and work with statistical graphics have come into fashion. 

Initially, data analysis was not the sort of thing a self-respecting academic would do. You'd ask your graduate students to generate a graphs or run an analysis for you. They would deliver a stack of paper and you would interpret the results. Then, in the 1970's John Tukey transformed data analysis into an honest intellectual exercise. In his text "Exploratory Data Analysis" he formulated a style of analysis that really informs programs like R. To Tukey, data analysis was a cyclic process involving multiple views of a data set. From simple numerical summaries to more advanced graphical representations. The data analyst would cycle between her ideas about patterns and relationships in the data, and computations that validate or contradict these ideas. Hypothesis leads to computation leads to hypothesis and so on.

<img src="http://ecx.images-amazon.com/images/I/41QMTYHJ4QL.jpg">

To Tukey, the more views you had of the data, the better. In the early 70s he developed a tool for interactive data visualization that generated an unbounded number of views. To get a sense of what I mean by interactive exploration, you can try a more recent program [ggobi](http://www.ggobi.org/). 

The struggle to find structure in data is really what statistics is all about. It's hard, however, to see data in more than 3 dimensions (scatterplots using x, y and z, say) and so the hunt was on to find a framework for teasing out patterns. This could be done with formal models (like regression) or other kinds of graphical displays that  extend the tried and true techniques like scatterplots. In the early 1990s this led to something called conditional plots or trellis displays (lattice in R). We'll see these also.

Finally, novelty. There was a desire to formalize and then expose the process by which basic graphics were made. So let's not start with a scatterplot and see what we can do with it, but instead break it down to its constitutent parts and develop a new framework for graphics from the bottom up. This is the spirit of the Grammar of Graphics or ggplot(2) in R.

OK, too much chatter. Let's make some pictures. We are going to use a data set associated with a [simple story from the NYT about the hardest places to live in the United States.](http://www.nytimes.com/2014/06/26/upshot/where-are-the-hardest-places-to-live-in-the-us.html?_r=0) It is county-level data. Quoting The UpShot, it consists of "six data points for each county in the United States: education (percentage of residents with at least a bachelor’s degree), median household income, unemployment rate, disability rate, life expectancy and obesity. We then averaged each county’s relative rank in these categories to create an overall ranking." Here's the data.

In [ ]:
hardest = read.csv("http://graphics8.nytimes.com/newsgraphics/2014/06/16/worst-places/d92796c59951a5cccb3ad6411f599ad302a5c4bd/unemployment.tsv",sep="\t",na.strings=c("No Data","#N/A"))
head(hardest)

In [ ]:
sample_n(hardest,10)

The data are formatted for display in the popup that appears when you hover over the New York Times' plot. It might be more useful to extract the state names separately. Here's how we pull out state using a regular explression and the str_replace() function in "stringr".

In [ ]:
str_replace(hardest$County,".*, (.*)","\\1")

And we can add the column to data set. 

In [ ]:
hardest = mutate(hardest,state=str_replace(County,".*, (.*)","\\1"))
head(hardest)

In [ ]:
table(hardest$state)

It looks like everything is here. Now, add a column called "cname" to the data set that is just the county name. Use a regular expression and "stringr" again.

In [ ]:
# Put your code here


Now, we have seen in this class that we might want to join several data sets together. Here is something from the Census that maps states to regions of the country. Have a look.

In [ ]:
regions = read.csv("https://www2.census.gov/programs-surveys/popest/datasets/2010-2015/state/asrh/scprc-est2015-18-pop-res.csv",na.strings="X",as.is=TRUE)
head(regions)

In [ ]:
sample_n(regions,10)

In [ ]:
table(regions$REGION)

Region 0 looks a little weird. What is that?

In [ ]:
filter(regions,REGION==0)

Finally, we join the two data sets, adding information about the region number to our hardest data set. We are keeping ust the state name and region number, but of course we could keep population or any other set of columns we might find useful later.

In [ ]:
head(select(regions,NAME,REGION))

In [ ]:
hardest = left_join(hardest,select(regions,NAME,REGION),by = c("state"="NAME"))
dim(hardest)

In [ ]:
sample_n(hardest,10)

To make this more interpretable, we can "map the values" 1 through 4 to the words describing the regions. We then add this new column to the data set using mutate.

In [ ]:
hardest = mutate(hardest,rname = mapvalues(REGION,c(1,2,3,4),c("Northeast","Midwest","South","West")))
sample_n(hardest,10)

And now our basic plots... here is a boxplot of education by region...

In [ ]:
boxplot(education~rname,data=hardest)

... or by state for a couple states.

In [ ]:
boxplot(education~state,
        data=filter(hardest, state %in% c("New York","Wisconsin","Texas","California")))

And two ways to make the same scatterplot.

In [ ]:
#plot(unemployment~education,data=hardest)
plot(hardest$education,hardest$unemployment)

Why two ways? plot() adapts its behavior to the kind of data you provide it. In R parlance it's called a "generic" function. We have already seen plots that change if the x-axis is date and time rather than simple numerical data. The command summary() also changes depending on whether you pass it a data frame or a vector of numbers or a vector of character strings. 

You recognize a generic function that adapts in this way because when you look at its printout, you see a call to a command "UseMethod" that is selecting what to do with the data you gave it.

In [ ]:
plot

In [ ]:
summary

In [ ]:
hist

In [ ]:
qqnorm

But today is about graphics and not the guts of R. Here is a simple histogram of the education data. Describe the distribution for me. 

In [ ]:
hist(hardest$education)

Using the normal distribution as a standard, we can make a more direct comparison using qqnorm().

In [ ]:
qqnorm(hardest$education)

These plots were designed to look "good" without much fuss. But "good" in the early 1970s is different than "good" now. You can tinker with an R plot and make it look as you like -- although it gets a little tedious.

R follows what's called a "painter's model" when crafting its basic plots. That is, you start with something simple and then add annotation, lines, points, etc. Here we add a number of different lines and points.

In [ ]:
plot(0:25,0:25,pch=0:25)
# and add a line say...
abline(5,3,col="red")
abline(20,-7,col="blue",lty=2)
abline(v=20.5)

In [ ]:
plot(0:25,0:25,pch=LETTERS)

Equivalently, we can simply add points to an existing plot; start with an “empty” or null plot...

In [ ]:
plot(c(0,25),c(0,25),type="n")

In [ ]:
plot(c(0,25),c(0,25),type="n")
# ... and add points
points(0:25,0:25,pch=0:25)

In [ ]:
# similar choices for line types
plot(c(0,6),c(0,6),type="n", xlab="",ylab="lty index")
# draw some horizontal lines of different types (note
# that lty 0 is blank!)
abline(h=0:6,lty=0:6)

In [ ]:
par(xpd=TRUE)
plot(hardest$education[1:3],hardest$unemployment[1:3],type="n")
text(hardest$education[1:3],hardest$unemployment[1:3],hardest$County[1:3])

Colors can be specified by name or RGB values, and can be created in a number of ways. The named colors were inherited from a windowing system for UNIX called X11.

In [ ]:
colors()

In [ ]:
plot(hardest$education,hardest$unemployment,pch=19,col="papayawhip")

In [ ]:
plot(hardest$education,hardest$unemployment,pch=19,col=rgb(0.5,1,0.2))

And there are constructor functions that give you a sequence of colors. topo.colors() gives you from blue to yellow. heat.colors() gives you from red to white. There's also rainbow() and cm.colors() (for cyan to magenta) and terrain.colors(). The barplot by region needs four colors, one for eac region.

In [ ]:
topo.colors(4)

In [ ]:
barplot(table(hardest$rname),col=topo.colors(4))

In [ ]:
barplot(table(hardest$rname),col=heat.colors(4))

Since R's initial offerings, the science of color choice has advanced considerably. Cynthia Brewer at Penn State has experimented with different color palettes and has come up with her [ColorBrewer](http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3) selector. It lets you specify whether you want a range of colors (to specify moving from low to high, say) or a set of colors that should look different from each other (for regions of the country, say).

In [ ]:
# a more sophisticated library
library(RColorBrewer)
# and have a look at the palettes available when you want, say, six different colors 
display.brewer.all(6)
display.brewer.all(10)

In [ ]:
# we can then select one we like...
pal = brewer.pal(6,"Blues")
# notice that it’s stored as hexadecimal RGB values -- the same format as returned
# by the function rgb()
pal
# [1] "#EFF3FF" "#C6DBEF" "#9ECAE1" "#6BAED6" "#3182BD" "#08519C"

In [ ]:
barplot(table(hardest$rname),col=pal)

Finally, a more detailed look at scatterplots. Here we plot two variables against each other and add a local regression or trend line. We talked about this kind of smoothing when we looked at the FBI UCR data. Here it is again, this time helping us make sense of the mash of data points at the left in this plot.  

In [ ]:
scatter.smooth(hardest$education,hardest$unemployment,col=5,pch=19,cex=0.1)

To make the most of our scatterplot tool, we might consider looking at more than two relationships at once. Here are all the pairwise relationships between education, disability and unemployment...

In [ ]:
pairs(select(hardest,education,disability, unemployment),pch=19,cex=0.5)

... and here we look at all possible pairs. This is what I meant by making the most of existing plots to help us see more.

In [ ]:
pairs(select(hardest,education,income,unemployment,disability,life,obesity),pch=19,cex=0.5)

Take a moment and go up to Cell->All Output->Clear. This will get rid of all the 3000 point plots we have floating around in this notebook. 

## Lattice

The second large graphics system in R is the "lattice" package.  It was introduced in the mid 1990s  with R's formal modeling language. This is where expresions like "unemployment~education" were introduced. For better or worse, it introduced a set of new names for the same old plots. plot() becomes xyplot(), and hist() becomes histogram(). We'll see that there's a payoff for all the new terms one has to learn. 

In [ ]:
library(lattice)

xyplot(unemployment~education,data=hardest)

Using the "|" in the modeling language means to "condition" on the variable that's indicated. So here, we make scatterplots of unemployment as a function of education, one for each region. 

In [ ]:
xyplot(unemployment~education|rname,data=hardest)

Lattice also introduced 3d scatterplots, or at least made them easy.

In [ ]:
cloud(unemployment~education*disability,data=hardest,pch=19,cex=0.5)

... and incorporated them into the whole conditioning framework.

In [ ]:
cloud(unemployment~education*disability|rname,data=hardest,pch=19,cex=0.5)

In [ ]:
histogram(~education|rname,data=hardest)

There is a lot in lattice, but you get the general idea. We hunt for structure through conditioning. 

## GGplot

The last major graphics system (aside from special graphics that come along with new models, say) is ggplot -- well, ggplot2. It comes from Lee Wilkinson's book "Grammar of Graphics." In it, Wilkinson tried to come up with a system for describing any plot. It was meant to not only recreate existing graphical forms, but allow for the creation of new ones. 

"ggplot2" was created by, well it's getting old now but, Hadley Wickham. You can read [the formal documentation](http://docs.ggplot2.org/current/index.html). There is also [a great cheetsheat for ggplot2](https://www.rstudio.com/wp-content/uploads/2016/11/ggplot2-cheatsheet-2.1.pdf) and I recommend this [tutorial for ggplot2](http://zevross.com/blog/2014/08/04/beautiful-plotting-in-r-a-ggplot2-cheatsheet-3/). 

In [ ]:
hardest = na.omit(hardest)

Let's start with a high-level plotting routine that comes with "ggplot2" -- it is called qplot() for quick plot. Here's a quick scatterplot followed by some elaborations.

In [ ]:
qplot(education,unemployment, data = hardest)

In [ ]:
qplot(education,unemployment, data = hardest, color = rname)

In [ ]:
qplot(education,unemployment, data = hardest, color = rname, size=income)

qplot() is designed to look as much like plot() as possible so you can add "xlab" and "ylab' and "main" to a plot, for example. Aside from scatterplots, it also creates barplots and histograms by specifying other "geom"'s -- bindings between data and geometric objects. Here we use "bar" for a barplot. Also available are "boxplot" and "dotplot" and "line" and "smooth" and "point" (which we have already seen). Here is [a nice summary of qplot()](http://ggplot2.org/book/qplot.pdf).

In [ ]:
qplot(rname, data = hardest, geom = "bar")

In [ ]:
qplot(education,unemployment, data = hardest, geom=c("point","smooth"))

qplot() is short for "quick plot" and it tries to be as much like plot() as it can. Let's take a second and work with the underlying "ggplot2" expressions so you get a sense of how this framework is constructed. (It would seem a little needless to give you another framework that looks like plot() -- what extra do you get by learning it?)

A plot begins with a map between data values and visual characteristics of our plot. 

In [ ]:
aes(x='obesity', y='life')

The command ggplot() takes a data set and constructs a plot using the aesthetics bindings and other information you provide. Here we get a set of x- and y-axes.

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))

The plot is blank because we have only set up the plot, made the bindings. Now we tell it what kind of geometric object we want to create. Points, lines, polygons...

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()

Notice that we added points by simply adding the specification for a geometric object. OK, let's break things down a little. The grammar of graphics builds on the following components of a plot.

* **data** in ggplot, data must be stored as a data frame
* **a coordinate system** describes 2-D space that data is projected onto (for example, Cartesian       coordinates, polar coordinates, map projections, and so on)
* **geoms** describe type of geometric objects that represent data (for example, points, lines,  polygons)
* **aesthetics** describe visual characteristics that represent data (for example, position, size,   color, shape, transparency, fill)
* **scales** for each aesthetic, describe how visual characteristic is converted to display values   (for example, log scales, color scales, size scales, shape scales, ...
* **stats** describe statistical transformations that typically summarize data (for example, counts, means, medians, regression lines)
* **facets** describe how data is split into subsets and displayed as multiple small graphs

geom_point() says that we want to render our x and y data as points. We can further adapt them by assigning colors and other features. 

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+
geom_point()+
coord_polar(theta="y")

You get the idea. We "add" components to a plot. Here we limit the y-range on the axis. A similar construction works for limiting the x-axis.

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+ylim(72,76)

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+scale_y_reverse()

We can add scales, adapting to the type of measurement we're plotting. Is it on the log-scale? Is it a date? Is it discrete? We can also add various statistical overlays to the plot...

In [ ]:
ggplot(hardest,aes(x =obesity,y=life))+geom_point()+stat_smooth(method=lm,color="blue")

In [ ]:
ggplot(hardest,aes(x =education,y=unemployment))+geom_point()+stat_smooth(method=lm,color="blue")

In [ ]:
ggplot(hardest,aes(x =education,y=unemployment))+geom_point()+stat_smooth(method=loess,color="blue")

Here we add a lot of components to the graphic...

In [ ]:
ggplot(hardest,aes(x=education,y=unemployment))+
    geom_point(color='lightblue')+
    stat_smooth(method="loess",color="black")+
    ggtitle("Life expectancy and obesity rates")+
    xlab("Percentage with College Education")+
    ylab("Unemployment rate")

... and here's a geometric component that represents a histogram...

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_histogram()

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_histogram(binwidth=0.5)

... or a smoothed histogram....

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_density()

... and using region names to give us different colors.

In [ ]:
ggplot(hardest,aes(x=obesity,color=rname))+geom_density()

The structure of "small multiples" offered by lattice is known here as faceting...

In [ ]:
ggplot(hardest,aes(x=obesity))+geom_density()+facet_wrap(~rname,scales='fixed')

A barplot geom object.

In [ ]:
ggplot(hardest,aes(x=rname))+geom_bar()

In [ ]:
ggplot(hardest,aes(x=obesity, y=life, color=rname)) + geom_point()

In [ ]:
ggplot(hardest,aes(x=obesity, y=life, color=rname)) + stat_smooth(method="loess",se=FALSE)

In [ ]:
ggplot(hardest,aes(x=obesity,y=life))+
    geom_point(color="lightblue")+
    stat_smooth(method="loess",color="black")+
    facet_wrap(~rname,scales='fixed')